In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from dna_data import DeepSEA_Dataset
import model

In [2]:
import sys
import logging
import datetime

log = logging.getLogger('snowdeer_log')
log.setLevel(logging.DEBUG)

log_file_name='log/'+sys.argv[-1].split('/')[-1].split('.')[-2].split('-')[-1]+'_'+datetime.datetime.now().strftime('%Y%m%d_%H%M%S')+'.log'
fileHandler = logging.FileHandler(log_file_name)
streamHandler = logging.StreamHandler()

formatter = logging.Formatter(' %(asctime)s [%(levelname)s] (%(filename)s:%(lineno)d) > %(message)s')
fileHandler.setFormatter(formatter)
streamHandler.setFormatter(formatter)

log.addHandler(fileHandler)
log.addHandler(streamHandler)

#log.debug('debug')

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Seq2SeqAttention(nn.Module):
    def __init__(self,
                input_size=4,
                hidden_layers = 2,
                hidden_size = 32,
                
                is_bidirectional = True,
                
                output_size = 919,
                
                dropout_keep = 0.8,
                ):
        
        super(Seq2SeqAttention, self).__init__()
        
        self.batch_first=False
        
        
        # LSTM Layer
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            num_layers = hidden_layers,
                            #batch_first=True,
                            dropout = dropout_keep,
                            bidirectional = is_bidirectional)
        
        # Dropout Layer
        self.dropout = nn.Dropout(dropout_keep)
        
        # Fully-Connected Layer
        self.fc = nn.Linear(
            hidden_size  * (1+is_bidirectional) * 2,
            output_size
        )
        
        
    def forward(self, x):        
        #embedded_sent = self.embeddings(x)

        lstm_out, (h_n,c_n) = self.lstm(x) 
        # (seq_len, batch, embed_size) ->
        # (seq_len, batch, num_directions * hidden_size),
        # ((hidden_layers * num_directions, batch, hidden_size),
        # (hidden_layers * num_directions, batch, hidden_size))
        print(h_n.shape)
        
        final_feature_map = self.dropout(h_n)
        # (hidden_layers * num_directions, batch, hidden_size)

        final_feature_map = torch.cat([final_feature_map[i,:,:] for i in range(final_feature_map.shape[0])], dim=1)
        # (hidden_layers * num_directions, batch, hidden_size) ->
        # (batch, hidden_layers * num_directions * hidden_size)

        final_out = self.fc(final_feature_map)
        # (batch, hidden_layers * num_directions * hidden_size) -> (output_size)
  
        return final_out

In [5]:
model=Seq2SeqAttention()
model.to(device)
#print(batch_X.shape)
#model(batch_X)

Seq2SeqAttention(
  (lstm): LSTM(4, 32, num_layers=2, dropout=0.8, bidirectional=True)
  (dropout): Dropout(p=0.8, inplace=False)
  (fc): Linear(in_features=128, out_features=919, bias=True)
)

In [7]:
dataloader_valid

NameError: name 'dataloader_valid' is not defined

In [6]:
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9,weight_decay=5e-07)
optimizer = optim.Adagrad(model.parameters(),weight_decay=5e-07)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5,verbose=1)
loss_function = nn.BCEWithLogitsLoss()

In [7]:
BATCH_SIZE=100
NUM_WORKERS=4

dataloader_valid = DataLoader(
    dataset=DeepSEA_Dataset('data/valid.mat','validxdata','validdata'), 
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS
)

(8000, 4, 1000) (8000, 919)


In [25]:
TOTAL_EPOCH=10

for epoch in range(TOTAL_EPOCH):
    train_loss_list=[]
    valid_loss_list=[]
    for i in range(10):
        # train data load
        try:
            del dataloader_train,dataset_train
        except:
            pass
        dataset_train=DeepSEA_Dataset('data/train_{}.pt'.format(i),'trainxdata','traindata')
        dataloader_train=DataLoader(dataset=dataset_train,shuffle=True,batch_size=BATCH_SIZE,num_workers=NUM_WORKERS)

        for batch_idx,(batch_X,batch_Y) in enumerate(dataloader_train):
            if model.batch_first==False:
                batch_X=batch_X.transpose(1,2).transpose(0,1)
            batch_X=batch_X.to(device) 
            batch_Y=batch_Y.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            output=model(batch_X)
            loss = loss_function(output, batch_Y)
            loss.backward()
            optimizer.step()  
            
            train_loss_list.append(loss.item())
            
            if batch_idx%100==99:
                log.info('epoch: {:d}, dataset_split: {:d}, batch: {:d}, progress: {:.2f}% loss: {:.3f}'.format(epoch,i,batch_idx,100*(i*len(dataloader_train)+batch_idx)/(10*len(dataloader_train)),np.mean(train_loss_list)))
                train_loss_list.clear()
        
        with torch.no_grad():
            for batch_idx, (batch_X, batch_Y) in enumerate(dataloader_valid):
                if model.batch_first==False:
                    batch_X = batch_X.transpose(1,2).transpose(0,1).to(device)
                else:
                    batch_X = batch_X.to(device)
                batch_Y = batch_Y.to(device)

                output = model(batch_X)
                loss = loss_function(output, batch_Y)
                valid_loss_list.append(loss.item())
                
            log.info('epoch: {:d}, dataset_split: {:d}, valid loss: {:.3f}'.format(epoch,i,np.mean(valid_loss_list)))
            valid_loss_list.clear()

 2019-12-30 14:20:42,258 [INFO] (<ipython-input-25-515f3a41de82>:32) > epoch: 0, dataset_split: 0, batch: 99, progress: 0.23% loss: 0.128


KeyboardInterrupt: 